In [1]:
trait RNG {
    def nextInt: (Int, RNG)
}

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-5DEVR1B:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1644072258610)
SparkSession available as 'spark'


defined trait RNG


In [2]:
case class SimpleRNG(seed: Long) extends RNG {
  def nextInt: (Int, RNG) = {
    val newSeed = (seed * 0x5DEECE66DL + 0xBL) & 0xFFFFFFFFFFFFL
    val nextRNG = SimpleRNG(newSeed)
    val n = (newSeed >>> 16).toInt
    (n, nextRNG)
  }
}

defined class SimpleRNG


In [3]:
val rng = SimpleRNG(42)

rng: SimpleRNG = SimpleRNG(42)


In [4]:
val (n1, rng2) = rng.nextInt

n1: Int = 16159453
rng2: RNG = SimpleRNG(1059025964525)


## 6-1

In [5]:
def nonNegativeInt(rng: RNG):(Int,RNG) = {
    val(n1,rng2) = rng.nextInt
    (if(n1 <0) -(n1+1) else n1,rng2)
}

nonNegativeInt: (rng: RNG)(Int, RNG)


In [6]:
nonNegativeInt(SimpleRNG(4))

res0: (Int, RNG) = (1538995,SimpleRNG(100859615679))


## 6-2

In [7]:
if(n1 < 0) -(n1+1) else n1

res1: Int = 16159453


In [8]:
def double(rng:RNG): (Double, RNG) = {
    val (n1, rng2) = nonNegativeInt(rng)
    (n1.toDouble/Int.MaxValue.toDouble,rng2)
}

double: (rng: RNG)(Double, RNG)


In [9]:
double(SimpleRNG(10))

res2: (Double, RNG) = (0.0017916266814766576,SimpleRNG(252149039181))


## 6-3

In [10]:
def intDouble(rng:RNG): ((Int,Double), RNG) = {
    val (n2, rng2) = nonNegativeInt(rng)
    val (n3, rng3) = double(rng2)
    ((n2,n3), rng3)
}

intDouble: (rng: RNG)((Int, Double), RNG)


In [11]:
def doubleInt(rng:RNG): ((Double,Int), RNG) = {
    val (n2, rng2) = double(rng)
    val (n3, rng3) = nonNegativeInt(rng2)
    ((n2,n3), rng3)
}

doubleInt: (rng: RNG)((Double, Int), RNG)


In [12]:
def double3(rng:RNG): ((Double,Double,Double),RNG) = {
    val (n2, rng2) = double(rng)
    val (n3, rng3) = double(rng2)
    val (n4, rng4) = double(rng3)
    ((n2,n3,n4),rng4)
}

double3: (rng: RNG)((Double, Double, Double), RNG)


## 6.4

In [54]:
def ints(count: Int)(rng:RNG): (List[Int],RNG) = {
    if (count <= 0)
      (List(), rng)
    else{
      val (x, r1)  = nonNegativeInt(rng)
      val (xs, r2) = ints(count - 1)(r1)
      (x :: xs, r2)
    }
}

ints: (count: Int)(rng: RNG)(List[Int], RNG)


In [55]:
ints(5)(SimpleRNG(10))

res9: (List[Int], RNG) = (List(3847489, 1334288366, 1486862010, 711662464, 1453296529),SimpleRNG(186231735346465))


In [13]:
type Rand[+A] = RNG => (A,RNG)

defined type alias Rand


In [14]:
val int: Rand[Int] = _.nextInt
def unit[A](a: A): Rand[A] = rng => (a, rng)
def map[A, B](s: Rand[A])(f: A => B): Rand[B] = {
    rng => {
      val (a, rng2) = s(rng)
      (f(a), rng2)
    }
  }

int: Rand[Int] = $Lambda$2222/0x00000008017dd840@4da82f3f
unit: [A](a: A)Rand[A]
map: [A, B](s: Rand[A])(f: A => B)Rand[B]


map 구현할때 rng가 뜬금 어디서 나왔다 싶었는데 <br>
크게 보면
rng => (f(a),rng2)) 라는 걸 알 수 있고 <br>
이는 Rand[B] 형태 (RNG => (B,RNG)) 라는걸 알 수 있다.

In [15]:
def nonNegativeEven:Rand[Int] = {
    map(nonNegativeInt)(i => i - i % 2)
}

nonNegativeEven: Rand[Int]


원래는 def foo(param): returnType 을 넣어주는게 일반적인 형태인데
Rand 자체를 type으로 등록해서 RNG => (A,RNG)로 표현이 되므로 위와 같이 쓰면
nonNegativeEven(rng:RNG):(A,RNG) 와 같은 뜻임

In [16]:
nonNegativeEven(SimpleRNG(10))

res3: (Int, RNG) = (3847488,SimpleRNG(252149039181))


In [17]:
def double:Rand[Double] = {
    map(nonNegativeInt)(i => i/Int.MaxValue.toDouble)
}

double: Rand[Double]


In [18]:
double(SimpleRNG(10))

res4: (Double, RNG) = (0.0017916266814766576,SimpleRNG(252149039181))


## 6.6

In [19]:
def map2[A,B,C](ra: Rand[A], rb: Rand[B])(f: (A,B) => C) : Rand[C] = {
    rng1 => {
        val (a,rng2) = ra(rng1)
        val (b,rng3) = rb(rng2)
        (f(a,b),rng3)
    }
}

map2: [A, B, C](ra: Rand[A], rb: Rand[B])(f: (A, B) => C)Rand[C]


In [43]:
def both[A,B](ra: Rand[A],rb: Rand[B]): Rand[(A,B)] = map2(ra,rb)((_,_))

both: [A, B](ra: Rand[A], rb: Rand[B])Rand[(A, B)]


In [44]:
val randIntDouble: Rand[(Int,Double)] = {
    both(nonNegativeInt,double)
}

randIntDouble: Rand[(Int, Double)] = $Lambda$2352/0x0000000801853840@2701daa8


In [45]:
randIntDouble(SimpleRNG(10))

res8: ((Int, Double), RNG) = ((3847489,0.6213264384406276),SimpleRNG(87443922374356))


In [65]:
val x = List(1,2,3)

x: List[Int] = List(1, 2, 3)


In [74]:
List.fill(x.size)(x.map(_*2).head)

res25: List[Int] = List(2, 2, 2)


## 6.7

Hard 어렵..
Hint => List도 결국 2개씩 나눠서 처리한다는 걸 생각하자
=> 1부터 10이면 f(1,f(2,f(3,4))) ...